# PGA Tour Data Retrieval
This notebook is for working with PGA_Data folder for retreiving revelant PGA Tour data that has been webscraped from [pgatour.com](https://www.pgatour.com/).

In [151]:
# General libraries
import pandas as pd
import numpy as np
from importlib import reload
import difflib

# Self made libraries
import pga_data.scrape
import pga_data.mapping

# Reload libraries
reload(pga_data.mapping)
reload(pga_data.scrape)

<module 'pga_data.scrape' from 'C:\\Users\\jborman\\Documents\\Python\\dfs_golf\\pga_data\\scrape.py'>

## Try to Extract Player IDs from pgatour.com
Webscrape to perform this task